In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [23]:
df_words = pd.read_csv('words_to_translate.csv', sep=';')
df_words_test = df_words[:5]
df_words_test

,words,nature,translation
0,turned it (thrice) in his handu,NaN,NaN
1,maddeningly,NaN,NaN
2,the (strain) of not laughing,n,NaN
3,finger raised (pedantically),NaN,NaN
4,twilight,n,NaN


In [20]:
def operation(row):

    word = row['words']
    pattern = r"\((.*?)\)"  # This regex pattern matches anything inside parentheses
    if '(' in word:
        word = re.findall(pattern, word)[0]
    print(word)
    translations = get_translation(word, row['nature'])
    for translation in translations:
        print(translation)
        print()
    print()
    print()
    return 'translated'

In [21]:
df_words_test['translation'] = df_words_test.apply(operation, axis=1)

turned it (thrice) in his hand
turned it (thrice) in his hand
['thrice', 'adv', 'dated(three times)', 'trois fois ', 'Edith was married thrice and widowed each time.', 'Elle a été mariée trois fois et a fini veuve à chaque fois.']



maddeningly
maddeningly
['maddeningly', 'adv', '(very annoyingly)', 'désespérément, horriblement ', 'I tried to coax the secret from Maya, but she was maddeningly tight-lipped.']



the (strain) of not laughing
the (strain) of not laughing
['strain', 'n', '(physical effort)', 'effort ', 'The strain of lifting too many boxes was too much for him.', "L'effort qu'il devait fournir pour porter tous ces cartons était trop grand."]

['strain', 'n', 'figurative(stress, pressure)', 'pression ', 'The strain of working too many hours really affected him.', "La pression de trop d'heures de travail l'affectait beaucoup."]

['strain', 'n', '(muscular injury)', 'élongation ', 'claquage ', 'The player was out with a muscle strain.', "Le joueur a dû quitter le terrain apr

C:\Users\charl\AppData\Local\Temp\ipykernel_18932\2704481587.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_words_test['translation'] = df_words_test.apply(operation, axis=1)


In [14]:
def get_translation(word, word_nature):



    r = requests.get('https://www.wordreference.com/enfr/'+word)
    soup = BeautifulSoup(r.text, 'html.parser')
    test_array = (soup.find_all('table', class_='WRD')[0].find_all('tr', class_=['even', 'odd']))

    translation_blocks = []
    block_temp = []
    for i in range(len(test_array)):
        if i != 0:
            if test_array[i].attrs.get('class')[0] != test_array[i - 1].attrs.get('class')[0]:
                translation_blocks.append(block_temp)
                block_temp = []
        block_temp.append(test_array[i].find_all('td'))
    translation_blocks.append(block_temp)

    array_translations = []
    array_temp = []

    for block in translation_blocks:
        if block[0][0].find('em'): # check for nature
            nature = block[0][0].find('em')
            nature.extract()
        if block[0][0].find('a'): # remove a tag from english
            block[0][0].find('a').extract()
        word = block[0][0].get_text()
        figurative = ''
        if block[0][1].find('i'):
            figurative = block[0][1].find('i').get_text()
        definition = figurative+''.join(block[0][1].find_all(string=True, recursive=False)).strip()
        array_temp.append(word.strip())
        array_temp.append(nature.get_text())

        traduction_context = block[0][1].find('span', class_='dsense')

        if definition != '':
            array_temp.append(definition.replace('\xa0', '').strip())
        for y in range(len(block)):
            if block[y][1].find('span', class_='dsense'):
                traduction_context = block[y][1].find('span', class_='dsense').get_text()
                if block[y][2].find('em'):
                    block[y][2].find('em').extract()
                if block[y][2].find('a'):
                    block[y][2].find('a').extract()
                traduction = block[y][2].get_text()
                array_temp.append(traduction_context+traduction)
            elif block[y][1].has_attr('class') and 'FrEx' in block[y][1]['class']: # example
                array_temp.append(block[y][1].get_text())
            elif block[y][1].has_attr('class') and 'ToEx' in block[y][1]['class']: # example
                array_temp.append(block[y][1].get_text())
            else:  #
                if block[y][2].find('em'):
                    block[y][2].find('em').extract()
                if block[y][2].find('a'):
                    block[y][2].find('a').extract()
                traduction = block[y][2].get_text()
                array_temp.append(traduction)
        array_translations.append(array_temp)
        array_temp = []

    return array_translations